# PyTorch Linear Regression using built-ins

In [1]:
# Import NumPy, PyTorch, torch.nn
import torch
import numpy as np
import torch.nn as nn

## Problem Statement

The model will predict crop yields for 2 different fruits: apples, oranges.
The input variables are average temperature, rainfall, humidity.

In [2]:
# Input data (temperature, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], 
                   [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], 
                   [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

Before building a model, *input* and *target* need to be converted to PyTorch tensors.

In [3]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Dataset, DataLoader

We'll create a **TensorDataset**, which allows to access rows from `inputs` and `targets` as tuples. We'll also create a **DataLoader**, to split the data into batches while training. It also lets us shuffle and sample the data.

In [7]:
# Import tensor dataset and data loader
from torch.utils.data import TensorDataset, DataLoader

In [13]:
# Define Dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [18]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 73.,  67.,  43.],
         [ 73.,  67.,  43.],
         [ 87., 134.,  58.]]), tensor([[ 22.,  37.],
         [103., 119.],
         [ 56.,  70.],
         [ 56.,  70.],
         [119., 133.]])]

## nn.Linear

Instead of initializing weights `w` and biases `b` manually, we can define the model using `nn.Linear`

In [69]:
#model = nn.Linear(in_features=3, out_features=2)
model = nn.Linear(3, 2)

In [71]:
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3756,  0.2712,  0.4979],
        [ 0.1683, -0.3638,  0.1932]], requires_grad=True)
Parameter containing:
tensor([0.0039, 0.1220], requires_grad=True)


##  Optimizer

Instead of manually manipulating weights and biases using gradients, we can use the optimizer `optim.SGD`

In [72]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Loss function

Instead of manually defining a loss function, we can use the built-in loss function `mse_loss`

In [74]:
# Import nn.functional
import torch.nn.functional as F

In [75]:
loss_fn = F.mse_loss

In [73]:
# Calculate loss

#preds = model(inputs)
#loss = loss_fn(preds, targets)

loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6116.9448, grad_fn=<MseLossBackward>)


## Train the model

We can train the model now. Let's define a utility function `fit` to train the model for a given number of epochs

In [76]:
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            # Predictions
            preds = model(xb)
            loss = loss_fn(preds, yb)
            
            # Gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
    print('Training loss: ', loss_fn(model(inputs), targets))

In [79]:
# Train the model for 100 epochs
fit(num_epochs=100, model=model, loss_fn=loss_fn, opt=opt)

Training loss:  tensor(1.6438, grad_fn=<MseLossBackward>)


In [80]:
preds = model(inputs)
print(preds)
print(targets)

tensor([[ 57.2367,  70.5099],
        [ 81.8333,  99.7453],
        [119.4393, 134.9564],
        [ 21.4475,  37.7853],
        [101.0556, 117.0878],
        [ 57.2367,  70.5099],
        [ 81.8333,  99.7453],
        [119.4393, 134.9564],
        [ 21.4475,  37.7853],
        [101.0556, 117.0878],
        [ 57.2367,  70.5099],
        [ 81.8333,  99.7453],
        [119.4393, 134.9564],
        [ 21.4475,  37.7853],
        [101.0556, 117.0878]], grad_fn=<AddmmBackward>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression from scratch 

Initialize weights and biases with random values

In [22]:
# Random initialization of w and b
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.3389, -1.5558,  0.0352],
        [-0.9144, -0.9678, -0.8755]], requires_grad=True)
tensor([ 0.8349, -1.7029], requires_grad=True)


The model is simply a function that performs a matrix multiplication of *input* and *w*, and adds the bias *b*.

In [23]:
# Define the model
def model(x):
    return x @ w.t() + b

In [24]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -77.1498, -170.9441],
        [-102.9817, -226.1129],
        [-176.1134, -261.7185],
        [ -30.1957, -168.9836],
        [-122.6715, -218.9905]], grad_fn=<AddBackward0>)


In [25]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


The model is pretty far off due to random initialization.

##  Loss function

In [40]:
# MSE
def mse(pred, act):
    diff = pred - act
    return torch.sum(diff * diff) / diff.numel()

In [41]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(66984.8594, grad_fn=<DivBackward0>)


In [ ]:
# Compute gradients w.r.t. w and b. The gradients are stored in w.grad() and b.grad()
loss.backward()

In [45]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.3389, -1.5558,  0.0352],
        [-0.9144, -0.9678, -0.8755]], requires_grad=True)
tensor([[-14606.4873, -17713.1016, -10469.0195],
        [-25205.6777, -27825.1719, -17094.0469]])


In [46]:
# Gradients for bias
print(b)
print(b.grad)

tensor([ 0.8349, -1.7029], requires_grad=True)
tensor([-178.0224, -301.3499])


In [47]:
# Reset gradients before moving forward
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Adjust weights and biases using gradient descent

1. Generate predictions
2. Calculate loss
3. Calculate gradients of loss w.r.t. weights and biases
4. Update weights and biases by subtracting a small quantity proportional to gradient
5. Reset gradients to zero

In [48]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -77.1498, -170.9441],
        [-102.9817, -226.1129],
        [-176.1134, -261.7185],
        [ -30.1957, -168.9836],
        [-122.6715, -218.9905]], grad_fn=<AddBackward0>)


In [49]:
# Calculate loss
loss = mse(preds, targets)
print(loss)

tensor(66984.8594, grad_fn=<DivBackward0>)


In [50]:
# Calculate gradient
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-14606.4873, -17713.1016, -10469.0195],
        [-25205.6777, -27825.1719, -17094.0469]])
tensor([-178.0224, -301.3499])


In [52]:
# Update weights and biases
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [54]:
print(w)
print(b)

tensor([[ 0.4849, -1.3786,  0.1399],
        [-0.6624, -0.6895, -0.7046]], requires_grad=True)
tensor([ 0.8367, -1.6999], requires_grad=True)


With the new weights and biases, the model should have a smaller loss

In [55]:
# Calculate new predictions and new loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(45691.1289, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, repeat the process of adjusting weights and biases using the gradients multiple times.

In [81]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [82]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(5.0539, grad_fn=<DivBackward0>)


In [83]:
# Print predictions
print(preds)

tensor([[ 57.2880,  70.2838],
        [ 82.4022,  98.7401],
        [118.0539, 137.3833],
        [ 20.9978,  38.3174],
        [102.4732, 114.6873]], grad_fn=<AddBackward0>)


In [69]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
